# Day 4: Camp Cleanup

# Part 1

Space needs to be cleared before the last supplies can be unloaded from the ships, and so several Elves have been assigned the job of cleaning up sections of the camp. Every section has a unique `ID number`, and each Elf is assigned a range of section IDs.

However, as some of the Elves compare their section assignments with each other, they've noticed that many of the assignments `overlap`. To try to quickly find overlaps and reduce duplicated effort, the Elves pair up and make a **big list of the section assignments for each pair** (your puzzle input).

For example, consider the following list of section assignment pairs:

```
2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8
```
For the first few pairs, this list means:

Within the first pair of Elves, the first Elf was assigned sections 2-4 (sections 2, 3, and 4), while the second Elf was assigned sections 6-8 (sections 6, 7, 8).
The Elves in the second pair were each assigned two sections.
The Elves in the third pair were each assigned three sections: one got sections 5, 6, and 7, while the other also got 7, plus 8 and 9.
This example list uses single-digit section IDs to make it easier to draw; your actual list might contain larger numbers. Visually, these pairs of section assignments look like this:

```
.234.....  2-4
.....678.  6-8

.23......  2-3
...45....  4-5

....567..  5-7
......789  7-9

.2345678.  2-8
..34567..  3-7

.....6...  6-6
...456...  4-6

.23456...  2-6
...45678.  4-8
```
Some of the pairs have noticed that one of their assignments fully contains the other. For example, 2-8 fully contains 3-7, and 6-6 is fully contained by 4-6. In pairs where one assignment fully contains the other, one Elf in the pair would be exclusively cleaning sections their partner will already be cleaning, so these seem like the most in need of reconsideration. In this example, there are 2 such pairs.

**In how many assignment pairs does one range fully contain the other?**





In [1]:
%load_ext nb_mypy

Version 1.0.4


In [2]:
from collections.abc import Generator
from pathlib import Path

INPUT_FILE = Path.cwd() / "inputs" / "day4.txt"
assert INPUT_FILE.exists(), INPUT_FILE


def range_from_str(s: str) -> range:
    i1, i2 = s.split("-")
    r = range(int(i1), int(i2))
    return r


def ingest_range_pairs(input_file: Path) -> Generator[tuple[range, range], None, None]:
    with open(input_file) as f_in:
        for line in f_in:
            s1, s2 = line.rstrip().split(",")
            yield range_from_str(s1), range_from_str(s2)


def full_overlap(r1: range, r2: range) -> bool:
    if r1.start >= r2.start and r1.stop <= r2.stop:
        return True
    if r2.start >= r1.start and r2.stop <= r1.stop:
        return True
    return False


overlap_count = 0
for i, pairs in enumerate(ingest_range_pairs(INPUT_FILE), start=1):
    if full_overlap(*pairs):
        overlap_count += 1
        # print(f"{i} Overlap {pairs}")

print(f"\n{overlap_count=}")


overlap_count=518


## Part Two
It seems like there is still quite a bit of duplicate work planned. Instead, the Elves would like to know the number of pairs that overlap at all.

In the above example, the first two pairs (2-4,6-8 and 2-3,4-5) don't overlap, while the remaining four pairs (5-7,7-9, 2-8,3-7, 6-6,4-6, and 2-6,4-8) do overlap:

5-7,7-9 overlaps in a single section, 7.
2-8,3-7 overlaps all of the sections 3 through 7.
6-6,4-6 overlaps in a single section, 6.
2-6,4-8 overlaps in sections 4, 5, and 6.
So, in this example, the number of overlapping assignment pairs is 4.

**In how many assignment pairs do the ranges overlap?**

In [3]:
def partial_overlap(r1: range, r2: range) -> bool:
    r1_set = set(r1)
    r2_set = set(r2)
    for range_, set_ in zip((r1, r2), (r1_set, r2_set)):
        set_.add(range_.stop)  # end of range should be included

    if r1_set.intersection(r2_set):
        return True
    return False


overlap_count = 0
for i, pairs in enumerate(ingest_range_pairs(INPUT_FILE), start=1):
    if partial_overlap(*pairs):
        overlap_count += 1
        # print(f"{i} Overlap {pairs}")

print(f"\n{overlap_count=}")


overlap_count=909


## Alternate Solution using custom objects

Would be more useful if you could do things like add and subtract intervals, but that's not needed for this exercise.

In [4]:
from __future__ import annotations


class Interval:
    _start: int
    _stop: int
    _range: range
    _set: set[int]

    def __init__(self, interval: str):
        if not isinstance(interval, str):
            raise TypeError(
                f"Expected interval string in the format '1-99', got {type(interval)}"
            )

        start, stop = interval.split("-")
        self._start = int(start)
        self._stop = int(stop)
        self._range = range(self._start, self._stop + 1)
        self._set = set(self._range)

    def __repr__(self) -> str:
        return f"Interval({self._start}, {self._stop})"

    def __eq__(self, other) -> bool:
        if not isinstance(other, self.__class__):
            return False
        return self._start == other._start and self._stop == other._stop

    def __contains__(self, other) -> bool:
        return self._set.issuperset(set(other))

    def __iter__(self):
        for i in self._range:
            yield i

    def intersection(self, other) -> set[int]:  # type: ignore[valid-type]
        return self._set.intersection(set(other))

In [5]:
a = Interval("1-10")
b = Interval("2-8")
A = Interval("1-10")
c = Interval("11-20")

assert a == A
assert a != object()
assert a != b
assert b in a
assert c not in a
assert [1, 2] in a

In [6]:
def ingest_intervals(
    input_file: Path,
) -> Generator[tuple[Interval, Interval], None, None]:
    with open(input_file) as f_in:
        for line in f_in:
            s1, s2 = line.rstrip().split(",")
            yield Interval(s1), Interval(s2)

In [7]:
full_overlap_count = 0
partial_overlap_count = 0
for i1, i2 in ingest_intervals(INPUT_FILE):
    if i1 in i2 or i2 in i1:
        full_overlap_count += 1

    if i1.intersection(i2):
        partial_overlap_count += 1

print(f"\n{full_overlap_count=}")
print(f"\n{partial_overlap_count=}")


full_overlap_count=518

partial_overlap_count=909
